# Getting Started with NVFlare (PyTorch)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/NVFlare/blob/main/examples/getting_started/pt/nvflare_pt_getting_started.ipynb)

NVFlare is an open-source framework that allows researchers and
data scientists to seamlessly move their machine learning and deep
learning workflows into a federated paradigm.

## Basic Concepts
At the heart of NVFlare lies the concept of collaboration through
"tasks." An FL controller assigns tasks (e.g., training on local data) to one or more FL clients, processes returned
results (e.g., model weight updates), and may assign additional
tasks based on these results and other factors (e.g., a pre-configured
number of training rounds). The clients run executors which can listen for tasks and perform the necessary computations locally, such as model training. This task-based interaction repeats
until the experiment’s objectives are met. 

<img src="../../../docs/resources/controller_executor_no_filter.png" alt="NVIDIA FLARE Controller and Executor" width=75% height=75% />

## Setup environment

Install nvflare and dependencies:

In [ ]:
! pip install --ignore-installed blinker
! pip install -r requirements.txt

If running in Google Colab, download the source code for this example:

In [ ]:
! npx degit -f NVIDIA/NVFlare/examples/hello-pt hello-pt

## Federated Averaging with NVFlare
Given the flexible controller and executor concepts, it is easy to implement different computing & communication patterns with NVFlare, such as [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) and [cyclic weight transfer](https://academic.oup.com/jamia/article/25/8/945/4956468). 

The controller's `run()` routine is responsible for assigning tasks and processing task results from the Executors. 

### Server Code
First, we provide a simple implementation of the [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) algorithm with NVFlare. 
The `run()` routine implements the main algorithmic logic. 
Subroutines, like `sample_clients()` and `scatter_and_gather_model()` utilize the communicator object, native to each Controller to get the list of available clients,
distribute the current global model to the clients, and collect their results.

The FedAvg controller implements these main steps:
1. FL server initializes an initial model using `self.load_model()`.
2. For each round (global iteration):
    - FL server samples available clients using `self.sample_clients()`.
    - FL server sends the global model to clients and waits for their updates using `self.send_model_and_wait()`.
    - FL server aggregates all the `results` and produces a new global model using `self.update_model()`.

```python
class FedAvg(BaseFedAvg):
    def run(self) -> None:
        self.info("Start FedAvg.")

        model = self.load_model()
        model.start_round = self.start_round
        model.total_rounds = self.num_rounds

        for self.current_round in range(self.start_round, self.start_round + self.num_rounds):
            self.info(f"Round {self.current_round} started.")
            model.current_round = self.current_round

            clients = self.sample_clients(self.num_clients)

            results = self.send_model_and_wait(targets=clients, data=model)

            aggregate_results = self.aggregate(results)

            model = self.update_model(model, aggregate_results)

            self.save_model(model)

        self.info("Finished FedAvg.")
```

### Client Code 
We take a CIFAR-10 example directly from [PyTorch website](https://github.com/pytorch/tutorials/blob/main/beginner_source/blitz/cifar10_tutorial.py) with some minor modifications, such as removing comments, move the network to [src/net.py](src/net.py), and add a main method and GPU support. The original code can be found at [cifar10_original.py](../../hello-world/ml-to-fl/pt/code/cifar10_original.py).

Now, we need to adapt this centralized training code to something that can run in a federated setting.

On the client side, the training workflow is as follows:
1. Receive the model from the FL server.
2. Perform local training on the received global model
and/or evaluate the received global model for model
selection.
3. Send the new model back to the FL server.

Using NVFlare's client API, we can easily adapt machine learning code that was written for centralized training and apply it in a federated scenario.
For a general use case, there are three essential methods to achieve this using the Client API :
- `init()`: Initializes NVFlare Client API environment.
- `receive()`: Receives model from the FL server.
- `send()`: Sends the model to the FL server.

With these simple methods, the developers can use the Client API
to change their centralized training code to an FL scenario with
five lines of code changes as shown below.
```python
    import nvflare.client as flare
    
    flare.init() # 1. Initializes NVFlare Client API environment.
    input_model = flare.receive() # 2. Receives model from the FL server.
    params = input_model.params # 3. Obtain the required information from the received model.
    
    # original local training code
    new_params = local_train(params)
    
    output_model = flare.FLModel(params=new_params) # 4. Put the results in a new `FLModel`
    flare.send(output_model) # 5. Sends the model to the FL server.  
```

The full client training script is saved in a separate file, e.g. [./src/cifar10_fl.py](./src/cifar10_fl.py) doing CNN training on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

## Run an NVFlare Job
Now that we have defined the FedAvg controller to run our federated compute workflow on the FL server, and our client training script to receive the global models, run local training, and send the results back to the FL server, we can put everything together using NVFlare's Job API.

#### 1. Define the initial model
First, we define the global model used to initialize the model on the FL server. See [src/net.py](src/net.py).

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
```

#### 2. Define a FedJob Recipe
 
 

In [ ]:
import argparse

from model import SimpleNetwork

from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv, add_experiment_tracking


def define_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument("--n_clients", type=int, default=2)
    parser.add_argument("--num_rounds", type=int, default=2)
    parser.add_argument("--batch_size", type=int, default=16)

    return parser.parse_args()


def main():
    args = define_parser()

    n_clients = args.n_clients
    num_rounds = args.num_rounds
    batch_size = args.batch_size

    recipe = FedAvgRecipe(
        name="hello-pt",
        min_clients=n_clients,
        num_rounds=num_rounds,
        initial_model=SimpleNetwork(),
        train_script="client.py",
        train_args=f"--batch_size {batch_size}",
    )
    add_experiment_tracking(recipe, tracking_type="tensorboard")

    env = SimEnv(num_clients=n_clients)
    run = recipe.execute(env)
    print()
    print("Result can be found in :", run.get_result())
    print("Job Status is:", run.get_status())
    print()


#### 6. Run Job


In [ ]:
! python job.py